# Convert from pytorch to C++/Matlab
Convert the net so it can be loaded by pytorch c++ inference. 

In [ ]:
# import torch
# from utils import *

# model_path = 'data/local/best_mse.pth'

# net = LiuqeNet()

# test_network_io(verbose=False)
# print(f'loading {model_path}')
# net.load_state_dict(torch.load(model_path, map_location='cpu'), strict=True)
# net.eval()

# net = net.double()


# torch.jit.script(net).save('net.pt')
# print('net.pt saved')

In [ ]:
# # create an example small ds and test inference on it to verify the conversion later
# import numpy as np
# from scipy.io import savemat
# ds = LiuqeDataset(EVAL_DS_PATH, verbose=False)

# ni = 5 # number of samples

# xs, _, rs, zs = ds[np.random.randint(0, len(ds), ni)] # inputs
# # convert to float64
# xs, rs, zs = xs.double(), rs.double(), zs.double()
# ys = net(xs, rs, zs) # outputs

# print(f'torch -> xs: {xs.shape}, ys: {ys.shape}, rs: {rs.shape}, zs: {zs.shape}')
# print(f'torch -> xs: {xs.dtype}, ys: {ys.dtype}, rs: {rs.dtype}, zs: {zs.dtype}')

# xs, ys, rs, zs = xs.numpy(), ys.detach().numpy().reshape(-1,NGR*NGZ), rs.numpy(), zs.numpy()
# print(f'numpy -> xs: {xs.shape}, ys: {ys.shape}, rs: {rs.shape}, zs: {zs.shape}')
# print(f'numpy -> xs: {xs.dtype}, ys: {ys.dtype}, rs: {rs.dtype}, zs: {zs.dtype}')

# # print the values to a txt files to compare them later
# with open(f'{TEST_DIR}/test_inference.txt', 'w') as f:
#     f.write('xs:\n')
#     for x in xs: 
#         f.write(' [')
#         for xi in x: f.write(f' {xi:+.4f}')
#         f.write(' ]\n')
#     f.write('ys:\n')
#     for y in ys: 
#         f.write(' [')
#         for i in range(NGR*NGZ): 
#             if i%NGR == 0 and i!=0: f.write(' ] [')
#             f.write(f' {y[i]:+.4f}')
#         f.write(' ]\n')
#     f.write('rs:\n')
#     for r in rs: 
#         f.write(' [')
#         for ri in r: f.write(f' {ri:+.4f}')
#         f.write(' ]\n')
#     f.write('zs:\n')
#     for z in zs: 
#         f.write(' [')
#         for zi in z: f.write(f' {zi:+.4f}')
#         f.write(' ]\n')

# # print the first ns samples of each vector
# ns = 5
# np.set_printoptions(precision=4, suppress=True, sign='+')
# print(f'x -> {xs[0,:ns]}')
# print(f'y -> {ys[0,:ns]}')
# print(f'r -> {rs[0,:ns]}')
# print(f'z -> {zs[0,:ns]}')

# # save the data to a .mat file for later comparison
# savemat(f'{TEST_DIR}/test_inference.mat', {'xs': xs, 'ys': ys, 'rs': rs, 'zs': zs})

In [ ]:
import torch
from torch import nn
class TestNet(nn.Module):
    def __init__(self):
        super(TestNet, self).__init__()
        self.lin = nn.Linear(2, 3)
    def forward(self, x):
        x = self.lin(x)
        return x
    
net = TestNet()
net = net.double()
net.eval()
x = torch.tensor([1,2], dtype=torch.float64).reshape(1,2)
y = net(x).detach()
print(f'input: {x}')
print(f'output: {y}')
print(f'input shape: {x.shape}')
print(f'output shape: {y.shape}')
print(f'input dtype: {x.dtype}')
print(f'output dtype: {y.dtype}')
x, y = x.numpy(), y.numpy()
print(f'numpy input: {x}')
print(f'numpy output: {y}')
print(f'numpy input shape: {x.shape}')
print(f'numpy output shape: {y.shape}')

# save to mat file
from scipy.io import savemat
savemat('test/test_inference.mat', {'x': x, 'y': y})

#save the model
torch.jit.script(net).save('net.pt')

input: tensor([[-1.0178,  0.7467]], dtype=torch.float64)
output: tensor([[0.0090, 0.8687, 0.9673]], dtype=torch.float64)
input shape: torch.Size([1, 2])
output shape: torch.Size([1, 3])
input dtype: torch.float64
output dtype: torch.float64
numpy input: [[-1.0178322   0.74669066]]
numpy output: [[0.00895858 0.86870863 0.96726778]]
numpy input shape: (1, 2)
numpy output shape: (1, 3)
